In [24]:
import pandas as pd
import numpy as np

In [25]:
v = pd.read_excel("../data/update_raw/jolts_labeled.xlsx", skiprows=3).drop(columns='Series ID')
v = v.melt(id_vars=['Industry', 'Series'], var_name='Date')
v = v[v['Series'] == 'job openings'].drop(columns='Series')
v = v.pivot(index=['Date'], columns='Industry', values='value').reset_index()
v.loc[:, 'Date'] = pd.to_datetime(v['Date'])
v.set_index('Date', inplace=True)
v.columns

Index(['Accomodation and food services', 'Arts, entertainment, and recreation',
       'Construction', 'Durable goods manufacturing',
       'Education and health services', 'Educational services', 'Federal',
       'Finance and insurance', 'Financial activities', 'Government',
       'Health care and social assistance', 'Information',
       'Leisure and hospitality', 'Manufacturing', 'Mining and logging',
       'Nondurable goods manufacturing', 'Other services',
       'Professional and business services',
       'Real estate and rental and leasing', 'Retail trade', 'State and local',
       'State and local government education',
       'State and local government, excluding education', 'Total nonfarm',
       'Total private', 'Trade, transportation, and utilities',
       'Transportation, warehousing, and utilities', 'Wholesale trade'],
      dtype='object', name='Industry')

In [26]:
# Can also keep hires
h = pd.read_excel("../data/update_raw/jolts_labeled.xlsx", skiprows=3).drop(columns='Series ID')
h = h.melt(id_vars=['Industry', 'Series'], var_name='Date')
h = h[h['Series'] == 'hires'].drop(columns='Series')
h = h.pivot(index=['Date'], columns='Industry', values='value').reset_index()
h.loc[:, 'Date'] = pd.to_datetime(h['Date'])
h.set_index('Date', inplace=True)
h.head()

Industry,Accomodation and food services,"Arts, entertainment, and recreation",Construction,Durable goods manufacturing,Education and health services,Educational services,Federal,Finance and insurance,Financial activities,Government,...,Real estate and rental and leasing,Retail trade,State and local,State and local government education,"State and local government, excluding education",Total nonfarm,Total private,"Trade, transportation, and utilities","Transportation, warehousing, and utilities",Wholesale trade
Date,,,,,,,,,,,,,,,,,,,,,
2000-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-04-01,847.0,151.0,458.0,244.0,486.0,64.0,35.0,152.0,222.0,334.0,...,70.0,827.0,299.0,148.0,150.0,5204.0,4871.0,1148.0,167.0,154.0
2002-04-01,700.0,128.0,431.0,245.0,500.0,62.0,48.0,147.0,221.0,340.0,...,74.0,738.0,291.0,141.0,150.0,4928.0,4588.0,1048.0,162.0,148.0
2003-04-01,639.0,148.0,461.0,189.0,470.0,73.0,25.0,134.0,232.0,309.0,...,98.0,714.0,285.0,146.0,139.0,4600.0,4291.0,971.0,118.0,139.0
2004-04-01,749.0,137.0,457.0,244.0,486.0,60.0,52.0,142.0,227.0,325.0,...,85.0,852.0,273.0,146.0,127.0,5152.0,4827.0,1188.0,154.0,182.0


In [27]:
u = pd.read_excel("../data/update_raw/CPS_Unemployment.xls", sheet_name='Monthly').melt(id_vars='DATE', var_name='Series', value_name='Unemployment')
crosswalk = pd.read_excel("../data/update_raw/CPS_Ind_Series_Crosswalk.xlsx")
u = pd.merge(crosswalk, u, on='Series', how='right').dropna(subset=['Industry']).drop(columns=['Series'])
u.rename(columns={'DATE': 'Date'}, inplace=True)
u['Industry'].unique()

array(['Government', 'Mining, quarrying, and oil and gas extraction',
       'Construction', 'Durable goods manufacturing',
       'Nondurable goods manufacturing', 'Wholesale and retail trade',
       'Transportation and utilities', 'Information',
       'Financial activities', 'Professional and business services',
       'Education and health services', 'Leisure and hospitality',
       'Other services'], dtype=object)

We can look at which sectors in the JOLTS data we need to recompute to match with the CPS data. First, the Mining sector is different from Mining and Logging. However, the logging sector employs around 70 thousand works while the mining sector employs around 700k workers. 

Second, I suspect wholesale and retail trade is the same as wholesale + retail trade. 

Third, I assume that `Transportation and Utilities` sector in the CPS data is the same as `Transportation, warehousing, and utilities` in the JOLTS data. 

In [28]:
v.loc[:, 'Wholesale and retail trade'] = v['Retail trade'] + v['Wholesale trade']
v.rename(columns={'Transportation, warehousing, and utilities': 'Transportation and utilities', 
                  'Mining and logging': "Mining, quarrying, and oil and gas extraction"}, inplace=True)
# Do the same with h
h.loc[:, 'Wholesale and retail trade'] = h['Retail trade'] + h['Wholesale trade']
h.rename(columns={'Transportation, warehousing, and utilities': 'Transportation and utilities', 
                  'Mining and logging': "Mining, quarrying, and oil and gas extraction"}, inplace=True)

Now we are all covered.

In [29]:
# Check if all u industries are covered
for i in u['Industry'].unique():
    print(i in v.columns)

True
True
True
True
True
True
True
True
True
True
True
True
True


In [30]:
v = v.reset_index().melt(id_vars='Date', var_name='Industry', value_name='Vacancy')
h = h.reset_index().melt(id_vars='Date', var_name='Industry', value_name='Hires')
uvh = pd.merge(u, v, on=['Date', 'Industry'], how='inner')
uvh = pd.merge(uvh, h, on=['Date', 'Industry'], how='inner')
uvh.loc[:, 'Tightness'] = uvh['Vacancy'] / uvh['Unemployment']

In [31]:
uvh

,Industry,Date,Unemployment,Vacancy,Hires,Tightness
0,Government,2000-01-01,430.0,NaN,NaN,NaN
1,Government,2000-02-01,409.0,NaN,NaN,NaN
2,Government,2000-03-01,311.0,NaN,NaN,NaN
3,Government,2000-04-01,269.0,NaN,NaN,NaN
4,Government,2000-05-01,370.0,NaN,NaN,NaN
...,...,...,...,...,...,...
3622,Other services,2022-11-01,165.0,314.0,214.0,1.903030
3623,Other services,2022-12-01,170.0,297.0,237.0,1.747059
3624,Other services,2023-01-01,209.0,359.0,210.0,1.717703
3625,Other services,2023-02-01,194.0,360.0,215.0,1.855670


In [32]:
# Merge short names
long_short_names = pd.read_excel("../data/update_raw/long_short_names_crosswalk.xlsx")
uvh = pd.merge(uvh, long_short_names, on='Industry')

In [33]:
uvh.dropna().head()

,Industry,Date,Unemployment,Vacancy,Hires,Tightness,short_names
11,Government,2000-12-01,365.0,548.0,342.0,1.501370,gov
12,Government,2001-01-01,463.0,482.0,337.0,1.041037,gov
13,Government,2001-02-01,298.0,479.0,366.0,1.607383,gov
14,Government,2001-03-01,355.0,445.0,346.0,1.253521,gov
15,Government,2001-04-01,369.0,490.0,334.0,1.327913,gov


In [34]:
uvh['Tightness'].describe()

count    3471.000000
mean        0.779143
std         0.721652
min         0.013817
25%         0.330739
50%         0.579186
75%         1.031153
max        18.000000
Name: Tightness, dtype: float64

In [35]:
uvh.dropna().sort_values(by='Tightness', ascending=False)

,Industry,Date,Unemployment,Vacancy,Hires,Tightness,short_names
555,"Mining, quarrying, and oil and gas extraction",2023-01-01,2.0,36.0,26.0,18.000000,mining
552,"Mining, quarrying, and oil and gas extraction",2022-10-01,4.0,41.0,21.0,10.250000,mining
348,"Mining, quarrying, and oil and gas extraction",2005-10-01,2.0,13.0,23.0,6.500000,mining
549,"Mining, quarrying, and oil and gas extraction",2022-07-01,6.0,35.0,25.0,5.833333,mining
2219,Information,2022-03-01,58.0,254.0,111.0,4.379310,info
...,...,...,...,...,...,...,...
689,Construction,2010-12-01,1749.0,46.0,378.0,0.026301,const
670,Construction,2009-05-01,1768.0,46.0,324.0,0.026018,const
668,Construction,2009-03-01,1979.0,47.0,328.0,0.023749,const
666,Construction,2009-01-01,1744.0,41.0,368.0,0.023509,const


In [36]:
uvh.dropna().sort_values(by='Tightness')

,Industry,Date,Unemployment,Vacancy,Hires,Tightness,short_names
669,Construction,2009-04-01,1737.0,24.0,331.0,0.013817,const
666,Construction,2009-01-01,1744.0,41.0,368.0,0.023509,const
668,Construction,2009-03-01,1979.0,47.0,328.0,0.023749,const
670,Construction,2009-05-01,1768.0,46.0,324.0,0.026018,const
689,Construction,2010-12-01,1749.0,46.0,378.0,0.026301,const
...,...,...,...,...,...,...,...
2219,Information,2022-03-01,58.0,254.0,111.0,4.379310,info
549,"Mining, quarrying, and oil and gas extraction",2022-07-01,6.0,35.0,25.0,5.833333,mining
348,"Mining, quarrying, and oil and gas extraction",2005-10-01,2.0,13.0,23.0,6.500000,mining
552,"Mining, quarrying, and oil and gas extraction",2022-10-01,4.0,41.0,21.0,10.250000,mining


In [37]:
uvh[uvh['Industry'] == 'Construction'].dropna()

,Industry,Date,Unemployment,Vacancy,Hires,Tightness,short_names
569,Construction,2000-12-01,580.0,233.0,414.0,0.401724,const
570,Construction,2001-01-01,836.0,241.0,495.0,0.288278,const
571,Construction,2001-02-01,826.0,253.0,486.0,0.306295,const
572,Construction,2001-03-01,683.0,188.0,575.0,0.275256,const
573,Construction,2001-04-01,596.0,226.0,458.0,0.379195,const
...,...,...,...,...,...,...,...
831,Construction,2022-10-01,419.0,398.0,351.0,0.949881,const
832,Construction,2022-11-01,393.0,348.0,337.0,0.885496,const
833,Construction,2022-12-01,443.0,488.0,380.0,1.101580,const
834,Construction,2023-01-01,720.0,283.0,387.0,0.393056,const


In [38]:
uvh.to_csv('../data/clean/uvh_updated.csv', index=False)

In [39]:
uvh.loc[:, 'Year'] = uvh['Date'].apply(lambda x: x.year)
uvh_annual = uvh.dropna().groupby(['Industry', 'short_names', 'Year']).aggregate({'Unemployment': sum, 'Vacancy': sum, 'Hires':sum}).reset_index()
uvh_annual.loc[:, 'Tightness'] = uvh_annual['Vacancy'] / uvh_annual['Unemployment']
uvh_annual.head()

,Industry,short_names,Year,Unemployment,Vacancy,Hires,Tightness
0,Construction,const,2000,580.0,233.0,414.0,0.401724
1,Construction,const,2001,7310.0,2207.0,5592.0,0.301915
2,Construction,const,2002,9596.0,1479.0,5225.0,0.154127
3,Construction,const,2003,9715.0,1328.0,5440.0,0.136696
4,Construction,const,2004,9228.0,1709.0,5562.0,0.185197


In [40]:
uvh_annual.to_csv('../data/clean/uvh_annual_updated.csv', index=False)

In [41]:
uvh_agg = uvh_annual.groupby('Year').aggregate({'Unemployment': sum, 'Vacancy': sum, 'Hires':sum})
uvh_agg.loc[:, 'Tightness'] = uvh_agg['Vacancy'] / uvh_agg['Unemployment']
uvh_agg

,Unemployment,Vacancy,Hires,Tightness
Year,,,,
2000,4516.0,5089.0,5426.0,1.126882
2001,71645.0,52201.0,62525.0,0.728606
2002,89264.0,41425.0,58192.0,0.464073
2003,92388.0,38866.0,56963.0,0.420682
2004,84386.0,43706.0,60502.0,0.517930
2005,78276.0,49293.0,63747.0,0.629733
2006,71953.0,55023.0,64751.0,0.764708
2007,72763.0,56252.0,63766.0,0.773085
2008,91817.0,45581.0,56420.0,0.496433


Note we don't have the full year of data for 2001 and 2023

The following code seeks to estimate parameters of the matching model using some variant of the regression
\begin{align*}
    \log H_{i,t} = \log \phi_i + \eta_i \log U_{i,t} + (1-\eta_i) \log V_{i,t} + \epsilon_{i,t}
\end{align*}
Where $\epsilon_t$ is some shock. While it is well understood that this regression suffers from endogeneity bias arrising from the behavior of agents on both sides of the market (see Borowczyk-Martings et al. 2013), we are simply looking for plausible calibration parameters for our model, and therefore keep the estimation purposefully simple. In the first instance, we constrain $\eta$ to equal $\frac{1}{2}$ and only estimate the sector specific matching efficiency under this assumption by least squares. 
\begin{align*}
    \log \widehat{\phi}_i = \frac{1}{T} \sum_{t=1}^{T}\left[\log h_{i,t} - \frac{1}{2}\left[\log U_{i,t} + \log V_{i,t}\right]\right]
\end{align*}

In [82]:
def matching_estimation1(df, eta):
    X = np.log(np.array(df[['Hires']].values)) - eta * np.log(np.array((df[['Unemployment']].values))) - (1-eta) * np.log(np.array(df[['Vacancy']].values))
    X = X[~np.isnan(X)]
    return np.exp(np.sum(X)/X.shape[0])

eta = 0.5
matching_param1 = uvh.groupby('short_names').apply(matching_estimation1, eta) 
df_matching_param1 = pd.DataFrame({'mathing_efficiency':matching_param1,'unemployment_elasticity':eta})
df_matching_param1

,mathing_efficiency,unemployment_elasticity
short_names,,
accom,1.119195,0.5
const,1.117432,0.5
dur,0.600169,0.5
edhealth,0.716745,0.5
fin,0.691572,0.5
gov,0.612414,0.5
info,0.642097,0.5
mining,1.016574,0.5
nondur,0.701085,0.5


In the second instance, we estimate the matching elasticity as well, assuming constant returns to scale holds. We can rewrite the linear model as
\begin{align*}
    \log H_{i,t} - \log V_{i,t} = \log \phi_i + \eta_i \left(\log U_{i,t}-\log V_{i,t}\right) + \epsilon_{i,t}
\end{align*}
Let $\theta = \begin{bmatrix} \log \phi_i & \eta_i \end{bmatrix}'$, then
\begin{align*}
    \widehat{\theta} = (X'X)^{-1} X'Y
\end{align*}
Where
\begin{align*}
    Y = \begin{bmatrix} 
        \log H_{i,1} - \log V_{i,1} \\
        \vdots \\
        \log H_{i,T} - \log V_{i,T}
    \end{bmatrix}, \, X = \begin{bmatrix}
        1 & \log U_{i,1} - \log V_{i,1} \\
        \vdots & \vdots \\
        1 & \log U_{i,T} - \log V_{i,T}
    \end{bmatrix}
\end{align*}

In [132]:
def matching_estimation2(df):
    log_H = np.log(np.array(df[['Hires']]))
    log_U = np.log(np.array(df[['Unemployment']]))
    log_U = log_U[~np.isnan(log_H)]
    log_V = np.log(np.array(df[['Vacancy']]))
    log_V = log_V[~np.isnan(log_H)]
    log_H = log_H[~np.isnan(log_H)]

    
    Y = np.ones((log_H.shape[0],1))
    Y[:,0] = log_H - log_V

    X = np.ones((log_H.shape[0],2))
    X[:,1] = log_U - log_V
    theta = np.linalg.inv(X.T @ X) @ (X.T @ Y)
    theta[0] = np.exp(theta[0])

    return theta.flatten()

matching_param2 = uvh.groupby('short_names').apply(matching_estimation2) 
matching_efficiency = np.ones(matching_param2.size)
unemployment_elasticity = np.ones(matching_param2.size)
for i in range(matching_param2.size):
    matching_efficiency[i] = matching_param2.iloc[i][0]
    unemployment_elasticity[i] = matching_param2.iloc[i][1]
df_matching_param2 = pd.DataFrame({'short_names':df_matching_param1.index.values,'matching_efficiency':matching_efficiency,
                                   'unemployment_elasticity':unemployment_elasticity})
df_matching_param2 = df_matching_param2.set_index('short_names')
df_matching_param2.to_csv('../data/clean/matching_param_estimates.csv')
df_matching_param2



,matching_efficiency,unemployment_elasticity
short_names,,
accom,1.185008,0.401161
const,1.105935,0.506507
dur,0.687520,0.364421
edhealth,0.702722,0.336267
fin,0.705406,0.329389
gov,0.640214,0.291264
info,0.702664,0.274926
mining,1.235578,0.262277
nondur,0.778797,0.391054


In [134]:

df_matching_param2.to_latex(float_format="%.3f")

/var/folders/dj/q47_fvg160bf035h7ccq_52m0000gn/T/ipykernel_45947/635184929.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_matching_param2.to_latex(float_format="%.3f")


'\\begin{tabular}{lrr}\n\\toprule\n{} &  matching\\_efficiency &  unemployment\\_elasticity \\\\\nshort\\_names &                      &                          \\\\\n\\midrule\naccom       &                1.185 &                    0.401 \\\\\nconst       &                1.106 &                    0.507 \\\\\ndur         &                0.688 &                    0.364 \\\\\nedhealth    &                0.703 &                    0.336 \\\\\nfin         &                0.705 &                    0.329 \\\\\ngov         &                0.640 &                    0.291 \\\\\ninfo        &                0.703 &                    0.275 \\\\\nmining      &                1.236 &                    0.262 \\\\\nnondur      &                0.779 &                    0.391 \\\\\nother       &                0.848 &                    0.441 \\\\\nprofserv    &                1.077 &                    0.372 \\\\\ntrade       &                1.009 &                    0.430 \\\\\ntrans